In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import tensorflow_addons as tfa

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
model1 = tf.keras.models.load_model('LSTM_new128.h5')
model2 = tf.keras.models.load_model('LSTM_25(128).h5')

In [ ]:
def evaluate():
    df = dft.copy()

    actual_close = dfact
    pred_close = predict_func(df)

    # Calculation of squared_error
    actual_close = np.array(actual_close)
    pred_close = np.array(pred_close)
    mean_square_error = np.mean(np.square(actual_close-pred_close))


    pred_prev = [df['Close'].iloc[-1]]
    pred_prev.append(pred_close[0])
    pred_curr = pred_close

    actual_prev = [df['Close'].iloc[-1]]
    actual_prev.append(actual_close[0])
    actual_curr = actual_close

    # Calculation of directional_accuracy
    pred_dir = np.array(pred_curr)-np.array(pred_prev)
    actual_dir = np.array(actual_curr)-np.array(actual_prev)
    dir_accuracy = np.mean((pred_dir*actual_dir)>0)*100

    print(f'Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}')
    return [mean_square_error, dir_accuracy]

In [ ]:
def predict_func(data):


    # data['Close'].fillna(-9999, inplace=True)
    # na_indices = data['Close'].index[data['Close'] == -9999]

    # for i in na_indices:
    #     data['Close'][i] = data['Close'][i-1]/2 + data['Close'][i+1]/2


    i=0

    while( pd.isna(data.at[i,'Close']) ):
        i=i+1

    # do backfilling till first non- null index as interpolation can't be done
    mask = data.index <= i
    data_filled = data.copy()
    data_filled.loc[mask] = data_filled.loc[mask].fillna(method='backfill')
    data=data_filled

    data=data.interpolate()

    df=data['Close']
    # df.interpolate(method="linear")

    df=np.array(df)
    # print(df)
    df=np.reshape(df,(50, 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    df = scaler.fit_transform(df)
    df=np.reshape(df,(1, 50, 1))

    predicted_1=model2.predict(df)
    predicted_1=np.array(predicted_1)

    df=np.reshape(df,(50, 1))
    df=np.append(df, predicted_1)

    df=np.reshape(df,(1, 51, 1))


    predicted_2=model2.predict(df[:,1:,:])
    predicted_2=np.array(predicted_2)

    # print(predicted_1)
    # print(predicted_2)

    df=np.reshape(df,(51,1))
    df=np.append(df, predicted_2)

    df=np.reshape(df,(52,1))
    df=scaler.inverse_transform(df)

    # print(df)

    return [df[-2][0], df[-1][0]]
    # return [0,0]

In [ ]:
if __name__== "__main__":
    result=evaluate()
    print(result[0])

In [ ]:
import yfinance as yf

In [ ]:
df = pd.read_csv('STOCK_INDEX.csv')
len = 150
dftata=df[len-50:len]
dfact = df["Close"][len:len+2]
dfact

2570    11930.34961
2571    11767.75000
Name: Close, dtype: float64

In [ ]:
if __name__== "__main__":
    data = pd.read_csv('STOCK_INDEX.csv')

    # df = yf.Ticker("NESTLEIND.NS")
    # Valid options are 1d, 5d, 1mo, 3mo, 6mo, 1y,
    # 2y, 5y, 10y and ytd.
    # df = df.history(period="5y")
    sum_rmse=0
    sum_acc=0
    for i in range(2400, 2600):
        dft=data[i-50:i]
        dfact=data['Close'][i:i+2]
        mean=np.mean(dft["Close"])
        evaluate()
    #   rmse=(res[0]**0.5)/mean
    #   print(rmse*100)
    #   dir_acc=res[1]
    #   sum_rmse+=rmse
    #   sum_acc+=dir_acc

    # length = 651
    # sum_rmse=sum_rmse/length
    # sum_acc=sum_acc/length
    # print(sum_rmse)
    # print(sum_acc)